## 웹 크롤링

### 인터넷 접속 라이브러리 추가

- urllib.request 모듈로 작업
- html 소스 로드로 별다른 문제 없음

In [11]:
from urllib.request import urlopen, Request

# 도시 별 날씨 검색함수
def get_weather(city):
    # 기상청 홈페이지 도시별 날씨 페이지
    url = 'https://www.weather.go.kr/w/obs-climate/land/city-obs.do'
    page = urlopen(url=url)

    text = page.read().decode('utf-8')
    text = text[text.find(f">{city}</a>"):]

    # 기온 가져오기
    for i in range(7):
        text = text[text.find('<td>')+1:]

    start = 3
    end = text.find('</td>')
    current_temp = (text[start:end])
    print(f'{city}의 현재 기온은 {current_temp}˚c 입니다.')

    # 습도 가져오기
    for i in range(3):
        text = text[text.find('<td>')+1:]

    start = 3
    end = text.find('</td>')
    current_humid = (text[start:end])
    print(f'{city}의 현재 습도는 {current_humid}% 입니다.')

get_weather('부산')

부산의 현재 기온은 7.1˚c 입니다.
부산의 현재 습도는 41% 입니다.


### OPEN API 크롤링

- 공공 데이터 포털
    - https://www.data.go.kr/index.do
    - 부산광역시 버스정보안내기 현황

``` python
    response = requests.get(total_url, verify=False)
```
- 문제점
    - https를 일반 request로 부르면 SSL오류 발생
    - 해결하려면 외부모듈 requests를 사용, verify = False 옵션을 추가로 지정해야 문제가 해결 됨
    - 아래 셀의 19번 라인

In [30]:
import requests
from urllib.parse import quote, unquote, urlencode # 한글을 URLencode 변환하는 함수
import json

# staitionName 정류소 이름
def getDataPortalSearch(stationName, type):
    api_url = 'https://apis.data.go.kr/6260000/BusanTblBusinfoeqStusService/getTblBusinfoeqStusInfo'
    queryString = "?" + urlencode(
        {
            'servicekey' : 'yR+IgD8xUGGLPLv5yhXB/1N8mrSZZShOsMcBdKjRIHdtT2R1/DttqVn95bmWMWkbyEbcz/J8qqJaoq5sRLTbbw==',
            'pageNumber' : "1",
            'numOfRows' : '10',
            'resultType' : type,
            'stationLoc' : stationName 
        }
    )

    total_url = api_url + queryString
    response = requests.get(total_url, verify=False)
    return response.text
try:
    result = getDataPortalSearch('신세계', 'json')
    json_data = json.loads(result)
    station_data = json_data['getTblBusinfoeqStusInfo']['body']['items']['item']


    for item in station_data:
        print(item)
except Exception as e:
    print('찾는 데이터가 없습니다.')

찾는 데이터가 없습니다.


c:\Source\StudyPython2023\venv\Lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'apis.data.go.kr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


###

### OpenAPI로 가져온 데이터 지도 표시

``` python
stop_str = '<h4>' + item['stationLoc'] + '</h4>' + item['stationNum'] + '<br>' + item['addr']
```

- 문제점
    - 딕셔너리 구조하고 문자열 포맷팅 f'{}' 호환 되지 않음
    - 구식 방법인 문자열 결합방식으로 해결해야 함

In [52]:
import folium

if len(station_data) > 0: # 정류소 중 젱리 첫번째 인덱스 정류소 위경도를 중심으로
    center_lat = station_data[0]['lat']
    center_lng = station_data[0]['lng']
    print(center_lat, center_lng)

m = folium.Map(location=[center_lat, center_lng], zoom_start=13)

# 전체 정류소 위치값 마커 표시
for item in station_data:
    stop_str = '<h4>' + item['stationLoc'] + '</h4>' + item['stationNum'] + '<br>' + item['addr']
    iframe = folium.IFrame(stop_str)
    popup = folium.Popup(iframe, min_width=200, max_width=200)

    folium.Marker(location=[item['lat'], item['lng']], popup=popup,
                  icon=folium.Icon(icon='pushpin')).add_to(m)

m

35.16967324 129.1317843


### BeautifulSoup (version 4)

웹 크롤링을 편하게 해주는 도구(라이브러리)

```shell
pip instiall beautifulsoup4
```

In [32]:
!pip install beautifulsoup4

     -------------------------------------- 129.4/129.4 kB 3.8 MB/s eta 0:00:00



[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: python.exe -m pip install --upgrade pip


#### 모듈 import

``` python
from bs4 import BeautifulSoup
```

In [46]:
import requests
from bs4 import BeautifulSoup

url = 'https://kin.naver.com/search/list.naver?query=%EC%A0%9C%EC%9E%84%EC%8A%A4+%EC%9B%B9+%EB%A7%9D%EC%9B%90%EA%B2%BD'
response = requests.get(url)

if response.status_code == 200:
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')       # html로 보기 좋게 'html.parser'
   # print(soup)
    title = soup.select_one('#s_content > div.section > ul > li:nth-child(1) > dl > dt > a')
    print(title.get_text())
else:
    print(f'Error : {response.status_code}')

제임스웹 우주망원경 질문이요


In [51]:
import requests
from bs4 import BeautifulSoup

url = 'https://kin.naver.com/search/list.naver?query=%EC%A0%9C%EC%9E%84%EC%8A%A4+%EC%9B%B9+%EB%A7%9D%EC%9B%90%EA%B2%BD'
response = requests.get(url)

if response.status_code == 200:
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')       # html로 보기 좋게 'html.parser'
    ul = soup.select_one('ul.basic1')
    titles = ul.select('li > dl > dt > a')
    for title in titles:
        print(title.get_text())
else:
    print(f'Error : {response.status_code}')

제임스웹 우주망원경 질문이요
최고고도 부탁드립니다. 제임스웹 망원경이
제임스 웹 망원경 우주사진 원본 어디서... 
제임스 웹 망원경 촬영 원리
제임스 웹 망원경
제임스 웹 우주망원경이 찍은 첫 사진은... 
제임스 웹 망원경의 종류 알려주세요
제임스 웹 우주망원경
제임스웹 망원경
제임스 웹 우주 망원경 제작 기간이
